# Build a stock market brief - S01E06-automate-the-brief
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Yahoo%20Finance/Build%20a%20stock%20market%20brief/S01E06-automate-the-brief.ipynb" target="_parent">
<img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/>
</a>

Bring everything together in an email to get your daily brief.

## Import necessary packages

In [1]:
import naas
import naas_drivers

## Set variables

In [2]:
stock = 'TSLA'
subject = "⏰ Reminder - Follow your favorite company : Tesla"

In [3]:
emails = open("users.txt", "r").read()

# Get stock data

In [4]:
data = naas_drivers.yahoofinance.get(stock, date_from=-200, moving_averages=[20,50])
data

## Calculate daily variations

In [5]:
data_display = data.copy()
data_display = data_display.sort_values(by="Date", ascending=False).reset_index(drop=True)
data_display

In [6]:
value = data_display.loc[0, "Close"]
value_last = data_display.loc[1, "Close"]

varv = value - value_last
varp = varv / abs(value_last) * 100

varv_diplay = '{:+.2f}'.format(varv)
varp_diplay = '{:+.2f}%'.format(varp)
value_display = '{:.2f}'.format(value)

print(value_display)
print(varv_diplay)
print(varp_diplay)

## Create chart

In [7]:
graph = naas_drivers.plotly.stock(data)

## Export chart in png and html

In [8]:
image_file = f"{stock}.png"
html_file = f"{stock}.html"
naas_drivers.plotly.export(graph, [html_file, image_file])

## Expose image and html

In [9]:
url_img = naas.assets.add(image_file)
url_html = naas.assets.add(html_file, params={"inline": True})

## Get score of the news 

In [10]:
%run S01E04-score-the-news.ipynb
sentiment_news

## Get prediction chart

In [11]:
%run S01E05-predict-the-future.ipynb

In [15]:
email_content = naas_drivers.html.generate( 
        display='iframe',
        title=f'Evolution of {stock} stock',
        image_0 = "https://i.pinimg.com/originals/e1/2a/67/e12a6721d52e079c5e33632f66ddb8a9.jpg",
        heading_1 = f'📈 Last close price: {value_display}' ,
        heading_2 = f'🎰 Variations: {varv_diplay} ({varp_diplay})' ,       
        heading_3 = f'😎 News sentiment: {sentiment_news}' ,
        
        button_Actual_200px=url_html,
        text_3 = "👉 Click on the button above to open dynamic chart." ,
        image= url_img,
        button_Prediction_200px= url_html_file_prediction,
        text_4 = "👉 Buy or Sell on <a href='https://www.etoro.com/markets/tsla'>Etoro</a>" ,
        image_future = url_image_file_prediction,
        heading_5 = "🔥 News hotlist",
        table = table_news_email,
        
)

# Send email to emails list

In [16]:
naas.notifications.send(emails, subject, email_content)

# Schedule this notebook and make it run every day at 9AM CET .

In [14]:
naas.scheduler.add(recurrence="0 9 * * *")